# Segmentation-clustering of neighborhood and Visualization

### Use k-means clustering and the geo spatial data extracted from Foursquare API to segment and cluster the neighborhoods in the city of New York, and visualize on map

In [2]:
import pandas as pd

In [3]:
## read table from webpage
## read_html always returns a list, add [0] to get the table - the df is the first item in list
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


Drop rows with Not Assigned Borough

In [4]:
df1=df[df.Borough != 'Not assigned']
df1

## OR df2 = df[df['Borough'] !='Not assigned'].reset_index(drop=True, )

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [5]:
for i in range(len(df1)):
    if df1['Neighbourhood'].iloc[i] =='Not assigned':
        df1['Neighbourhood'].iloc[i]= df1['Borough'].iloc[i]

In [6]:
df1.shape

(103, 3)

## GeoData

In [7]:
geo=pd.read_csv("http://cocl.us/Geospatial_data")
geo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Combine Geo data and neighbourhood dataframe

In [8]:
df_main=df1.join(geo.set_index('Postal Code'), on='Postal Code')
df_main

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
165,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Clustering

In [9]:
import json # library to handle JSON files
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [10]:
#!conda install -c conda-forge geopy --yes 
# uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


In [11]:
#!conda install -c conda-forge folium=0.5.0 --yes 
# uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library


### Map

In [16]:
address = "Toronto, ON"

## nominatim: search OSM data by name and address and to generate synthetic addresses of OSM points
## should specify a user_agent. The Nominatum service runs on donated servers which have limited capacity, 
## so specifying a user-agent field allows Open Street Map to track more easily who is using their service. 

geolocator = Nominatim(user_agent="toronto_explorer")

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [46]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

In [18]:
#Mark boroughs on map
for lat, lon, borough, neighbourhood in zip(
        df_main['Latitude'], 
        df_main['Longitude'], 
        df_main['Borough'], 
        df_main['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)  

map_toronto

Area of interest: borough of York

In [19]:
df_yk = df_main[df_main['Borough'].str.contains("York")].reset_index(drop=True)
df_yk.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937


Map for York area

In [20]:
# create map of York using latitude and longitude values
address_yk='York, Toronto'
location_yk = geolocator.geocode(address_yk)
latitude_yk = location.latitude
longitude_yk = location.longitude

map_yk = folium.Map(location=[latitude_yk, longitude_yk], zoom_start=10)

In [21]:
#Mark boroughs on map
for lat, lon, borough, neighbourhood in zip(
        df_yk['Latitude'], 
        df_yk['Longitude'], 
        df_yk['Borough'], 
        df_yk['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='',
        fill_opacity=0.5,
        parse_html=False).add_to(map_yk)  

map_yk

FourSquare 

In [22]:
# The code was removed by Watson Studio for sharing.

In [23]:
# first neighborhood
n_name=df_yk.loc[0, 'Neighbourhood']
n_lat = df_yk.loc[0, 'Latitude'] # neighborhood latitude value
n_lon = df_yk.loc[0, 'Longitude'] # neighborhood longitude value
print('Latitude and longitude values of {} are {}, {}.'.format(n_name, 
                                                               n_lat, 
                                                               n_lon))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


Nearby venues of york

In [24]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_yk, longitude_yk, VERSION, radius, LIMIT)


In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd86078eaedc31c263ee829'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 89,
  'suggestedBounds': {'ne': {'lat': 43.6579817045, 'lng': -79.37772678059432},
   'sw': {'lat': 43.6489816955, 'lng': -79.39014261940568}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          'lng'

format nearby venues into df

In [26]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Indigo,Bookstore,43.653515,-79.380696
3,Chatime 日出茶太,Bubble Tea Shop,43.655542,-79.384684
4,CF Toronto Eaton Centre,Shopping Mall,43.654447,-79.380952


In [28]:
nearby_venues.shape

(89, 4)

get nearby venues for all neighborhoods in York

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:

yk_venues = getNearbyVenues(names=df_yk['Neighbourhood'],
                                   latitudes=df_yk['Latitude'],
                                   longitudes=df_yk['Longitude'])

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Parkview Hill, Woodbine Gardens
Glencairn
Don Mills
Woodbine Heights
Humewood-Cedarvale
Caledonia-Fairbanks
Leaside
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Del Ray, Mount Dennis, Keelsdale and Silverthorn
Humberlea, Emery
Willowdale, Willowdale East
Downsview
Runnymede, The Junction North
Weston
York Mills West
Willowdale, Willowdale West


In [31]:
yk_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22,22,22
Caledonia-Fairbanks,4,4,4,4,4,4
"Del Ray, Mount Dennis, Keelsdale and Silverthorn",5,5,5,5,5,5
Don Mills,23,23,23,23,23,23
Downsview,16,16,16,16,16,16
"East Toronto, Broadview North (Old East York)",3,3,3,3,3,3
"Fairview, Henry Farm, Oriole",70,70,70,70,70,70


In [32]:
print('There are {} uniques categories.'.format(len(yk_venues['Venue Category'].unique())))

There are 121 uniques categories.


Analysis

In [33]:
# one hot encoding
yk_onehot = pd.get_dummies(yk_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
yk_onehot['Neighborhood'] = yk_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [yk_onehot.columns[-1]] + list(yk_onehot.columns[:-1])
yk_onehot = yk_onehot[fixed_columns]

yk_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,...,Thai Restaurant,Theater,Toy / Game Store,Trail,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:

yk_grouped = yk_onehot.groupby('Neighborhood').mean().reset_index()
yk_grouped.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bagel Shop,Bakery,...,Thai Restaurant,Theater,Toy / Game Store,Trail,Turkish Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Women's Store,Yoga Studio
0,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
1,Bayview Village,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
2,"Bedford Park, Lawrence Manor East",0.0,0.0,0.045455,0.0,0.0,0.0,0.0,0.0,0.0,...,0.045455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
3,Caledonia-Fairbanks,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.00,0.0


Print each neighborhood with top 5 venues

In [35]:
num = 5

for hood in yk_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = yk_grouped[yk_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
                venue  freq
0         Coffee Shop  0.10
1                Bank  0.10
2  Frozen Yogurt Shop  0.05
3      Sandwich Place  0.05
4  Chinese Restaurant  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1   Chinese Restaurant  0.25
2                 Café  0.25
3                 Bank  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.09
1         Coffee Shop  0.09
2      Sandwich Place  0.09
3    Greek Restaurant  0.05
4             Butcher  0.05


----Caledonia-Fairbanks----
           venue  freq
0           Park  0.50
1           Pool  0.25
2  Women's Store  0.25
3  Luggage Store  0.00
4       Pharmacy  0.00


----Del Ray, Mount Dennis, Keelsdale and Silverthorn----
                venue  freq
0      Sandwich Place   0.2
1                 Bar   0.2
2      Discount Store   0.2
3          Restaurant   0

In [36]:
# put into pandas
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
# df for top 10 venues
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = yk_grouped['Neighborhood']

for ind in np.arange(yk_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(yk_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Fried Chicken Joint,Ice Cream Shop,Mobile Phone Shop,Chinese Restaurant,Frozen Yogurt Shop,Pharmacy,Pizza Place,Deli / Bodega
1,Bayview Village,Bank,Chinese Restaurant,Japanese Restaurant,Café,Yoga Studio,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Deli / Bodega
2,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Greek Restaurant,Grocery Store,Indian Restaurant,Juice Bar,Liquor Store,Locksmith,Comfort Food Restaurant
3,Caledonia-Fairbanks,Park,Women's Store,Pool,Yoga Studio,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio
4,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",Turkish Restaurant,Bar,Sandwich Place,Discount Store,Restaurant,Yoga Studio,Deli / Bodega,Dim Sum Restaurant,Dessert Shop,Department Store


## Clustering

In [38]:
kclusters = 5

yk_grouped_clustering = yk_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(yk_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 3, 1, 1, 1, 3, 1, 1], dtype=int32)

In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)



In [40]:
yk_merged = df_yk

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
yk_merged = yk_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

yk_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Park,Food & Drink Shop,Yoga Studio,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Deli / Bodega
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Pizza Place,French Restaurant,Portuguese Restaurant,Intersection,Hockey Arena,Coffee Shop,Fried Chicken Joint,Dim Sum Restaurant,Comfort Food Restaurant,Furniture / Home Store
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Accessories Store,Boutique,Furniture / Home Store,Event Space,Coffee Shop,Gift Shop,Women's Store,Vietnamese Restaurant,Athletics & Sports
3,M3B,North York,Don Mills,43.745906,-79.352188,1.0,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Chinese Restaurant,Discount Store,Restaurant,Italian Restaurant,Dim Sum Restaurant,Sporting Goods Shop
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1.0,Pizza Place,Gym / Fitness Center,Bus Line,Breakfast Spot,Intersection,Athletics & Sports,Bank,Gastropub,Pharmacy,Pet Store


In [41]:
yk_merged.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Park,Food & Drink Shop,Yoga Studio,Dog Run,Construction & Landscaping,Convenience Store,Cosmetics Shop,Curling Ice,Dance Studio,Deli / Bodega
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Pizza Place,French Restaurant,Portuguese Restaurant,Intersection,Hockey Arena,Coffee Shop,Fried Chicken Joint,Dim Sum Restaurant,Comfort Food Restaurant,Furniture / Home Store
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Accessories Store,Boutique,Furniture / Home Store,Event Space,Coffee Shop,Gift Shop,Women's Store,Vietnamese Restaurant,Athletics & Sports
3,M3B,North York,Don Mills,43.745906,-79.352188,1.0,Gym,Coffee Shop,Japanese Restaurant,Beer Store,Chinese Restaurant,Discount Store,Restaurant,Italian Restaurant,Dim Sum Restaurant,Sporting Goods Shop
4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1.0,Pizza Place,Gym / Fitness Center,Bus Line,Breakfast Spot,Intersection,Athletics & Sports,Bank,Gastropub,Pharmacy,Pet Store


In [42]:

yk_merged = yk_merged[yk_merged["Cluster Labels"].notna()]

In [45]:
# create map
map_clusters = folium.Map(location=[latitude_yk, longitude_yk], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(yk_merged['Latitude'], yk_merged['Longitude'], yk_merged['Neighbourhood'], yk_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters